In [1]:
import pandas as pd
import os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [10]:
import importlib

package_name = "mlflow"
try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ModuleNotFoundError:
    print(f"{package_name} not found. Installing...")
    %pip install {package_name}


mlflow is already installed.


In [11]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [12]:
df_train = read_dataframe('/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-02.parquet')

In [13]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [14]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [17]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-exp-2")

<Experiment: artifact_location='/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week2/mlruns/2', creation_time=1679093170978, experiment_id='2', last_update_time=1679093170978, lifecycle_stage='active', name='nyc-taxi-exp-2', tags={}>

In [18]:
with mlflow.start_run():

    mlflow.set_tag("developer","bt")
    mlflow.set_tag("model","Lasso")

    mlflow.log_param("train data","/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-01.parquet")
    mlflow.log_param("test data","/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-02.parquet")

    alpha = 0.001

    mlflow.log_param("alpha",alpha)

    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train,y_train)

    y_pred = lasso.predict(X_val)

    rmse = mean_squared_error(y_val,y_pred,squared=False)

    mlflow.log_metric("rmse",rmse)